In [1]:
import torch

# Create a tensor
x = torch.tensor([1, 2, 3])
y = torch.ones_like(x)  # Creates a tensor of ones with the same shape as x

# Basic tensor operations
z = x + y  # Element-wise addition
w = torch.matmul(x, y)  # Matrix multiplication (dot product if 1D)


/Users/s/PhD/gitrepo/Torch-Practice/venv/lib/python3.13/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
# Example with autograd
x = torch.tensor([1.0, 2.0], requires_grad=True)
y = x**2
z = y.sum()

# Compute gradients
z.backward()

# Access gradients
print(x.grad)  # Gradient of z with respect to x


tensor([2., 4.])


In [3]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(10, 5)  # Input: 10 features, Output: 5 features
        self.layer2 = nn.ReLU()
        self.layer3 = nn.Linear(5, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x


In [4]:
import torch.optim as optim

# Model and optimizer
model = SimpleNN()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Zero the gradients, perform a backward pass, and update weights
optimizer.zero_grad()  # Clears old gradients
output = model(torch.randn(1, 10))  # Example input
loss = output.sum()  # Just an example loss function
loss.backward()
optimizer.step()  # Updates parameters


In [6]:
import gpytorch

class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.RBFKernel()
        self.likelihood = likelihood

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Training a GP model with GPytorch involves defining a likelihood and optimizing the model parameters using PyTorch's optimizers.


In [7]:
import torch
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf

# Define some data points and a GP model
train_x = torch.rand(10, 1)  # 10 data points, 1 feature each
train_y = torch.sin(train_x)

# Build a single-task GP model
gp = SingleTaskGP(train_x, train_y)

# Define an acquisition function (e.g., Expected Improvement)
# and optimize it to find the next sample
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

acqf = ExpectedImprovement(gp, best_f=train_y.max())
next_point, _ = optimize_acqf(acqf, bounds=torch.tensor([[0.], [1.]]), num_restarts=5, raw_samples=20)


/Users/s/PhD/gitrepo/Torch-Practice/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/_4/f_rgwft14s14_j1xnpz_zn7c0000gn/T/ipykernel_12979/3905559278.py:10: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  gp = SingleTaskGP(train_x, train_y)
/Users/s/PhD/gitrepo/Torch-Practice/venv/lib/python3.13/site-packages/botorch/acquisition/analytic.py:331: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fix

TypeError: optimize_acqf() missing 1 required positional argument: 'q'

In [4]:
import torch
from botorch.models import SingleTaskGP
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

# Define some data points and a GP model
train_x = torch.rand(10, 1)  # 10 data points, 1 feature each
train_y = torch.sin(train_x)

# Build a single-task GP model
gp = SingleTaskGP(train_x, train_y)

# Define an acquisition function (e.g., Expected Improvement)
acqf = ExpectedImprovement(gp, best_f=train_y.max())

# Specify the number of candidates to optimize over
q = 1  # For single candidate optimization

#define boudnds
bounds = torch.tensor([[0.],[1.]])

# Optimize the acquisition function to find the next sample
next_point, _ = optimize_acqf(
    acqf, 
    bounds=bounds,  #pass
    num_restarts=5,  # Number of random restarts for optimization
    raw_samples=20,  # Number of raw candidates to sample before optimization
    q=q  # Number of candidates to optimize over at once
)

print(next_point)


tensor([[1.]])


/var/folders/_4/f_rgwft14s14_j1xnpz_zn7c0000gn/T/ipykernel_13358/1056962898.py:11: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  gp = SingleTaskGP(train_x, train_y)
/Users/s/PhD/gitrepo/Torch-Practice/venv/lib/python3.13/site-packages/botorch/acquisition/analytic.py:331: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)


In [5]:
import torch
from botorch.models import SingleTaskGP
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

# Define some data points and a GP model
train_x = torch.rand(10, 1)  # 10 data points, 1 feature each
train_y = torch.sin(train_x)

# Build a single-task GP model
gp = SingleTaskGP(train_x, train_y)

# Define an acquisition function (e.g., Expected Improvement)
acqf = ExpectedImprovement(gp, best_f=train_y.max())

# Specify the number of candidates to optimize over
q = 1  # For single candidate optimization

# Define the bounds of the search space (ensure correct shape: 2 x 1)
bounds = torch.tensor([[0.], [1.]])  # Lower and upper bounds in the 1D space

# Optimize the acquisition function to find the next sample
next_point, _ = optimize_acqf(
    acqf, 
    bounds=bounds,  # Pass bounds here
    num_restarts=5,  # Number of random restarts for optimization
    raw_samples=20,  # Number of raw candidates to sample before optimization
    q=q  # Number of candidates to optimize over at once
)

print(next_point)


tensor([[1.]])


/var/folders/_4/f_rgwft14s14_j1xnpz_zn7c0000gn/T/ipykernel_13358/782265510.py:11: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  gp = SingleTaskGP(train_x, train_y)
/Users/s/PhD/gitrepo/Torch-Practice/venv/lib/python3.13/site-packages/botorch/acquisition/analytic.py:331: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)
